<a href="https://colab.research.google.com/github/angelouok/kwizera-angelo-derrick-2409000145/blob/main/ADVANCED%20WEB%20CHECK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import smtplib
import logging
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# List of URLs to monitor
URLS_TO_CHECK = [
    "https://uok.ac.rw",
    "https://example.com",
    "https://another-website.com"
]

# Email alert configuration
EMAIL_SENDER = "YOUR_EMAIL@gmail.com"
EMAIL_PASSWORD = "YOUR_PASSWORD"  # Use App Password for security
EMAIL_RECEIVER = "RECEIVER_EMAIL@gmail.com"
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

# Logging configuration
LOG_FILE = "website_monitor.log"
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")


def send_email_alert(url, error_message):
    """Send an email notification when a website goes down."""
    try:
        msg = MIMEMultipart()
        msg["From"] = EMAIL_SENDER
        msg["To"] = EMAIL_RECEIVER
        msg["Subject"] = f"🚨 ALERT: {url} is DOWN!"

        body = f"⚠️ The website {url} is unreachable.\nError: {error_message}\nPlease check immediately."
        msg.attach(MIMEText(body, "plain"))

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())

        logging.info(f"Email alert sent for {url}.")
    except Exception as e:
        logging.error(f"Failed to send email alert: {e}")


def check_website(url):
    """Check website status and log failures."""
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    try:
        response = requests.get(url, headers=headers, timeout=5)

        if response.status_code == 200:
            return f"✅ {url} is UP (Status: {response.status_code})"
        else:
            error_message = f"❌ {url} returned error (Status: {response.status_code})"
            logging.warning(error_message)
            send_email_alert(url, error_message)
            return error_message

    except requests.exceptions.Timeout:
        error_message = f"⏳ {url} is unreachable (Timeout)"
    except requests.exceptions.ConnectionError:
        error_message = f"🔌 {url} is unreachable (Connection Error)"
    except requests.exceptions.RequestException as e:
        error_message = f"⚠️ {url} is unreachable (Error: {e})"

    logging.error(error_message)
    send_email_alert(url, error_message)
    return error_message


def monitor_websites(interval=300):
    """Periodically check multiple websites and log failures."""
    while True:
        print("\n🔍 Running Website Checks...")
        for url in URLS_TO_CHECK:
            status = check_website(url)
            print(status)

        print(f"🕒 Waiting {interval / 60} minutes before next check...\n")
        time.sleep(interval)  # Wait before checking again


if __name__ == "__main__":
    monitor_websites()  # Runs the monitoring loop every 5 minutes



🔍 Running Website Checks...
✅ https://uok.ac.rw is UP (Status: 200)


ERROR:root:🔌 https://another-website.com is unreachable (Connection Error)
ERROR:root:Failed to send email alert: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 4fb4d7f45d1cf-5e45a8b8b53sm1519626a12.19 - gsmtp')


✅ https://example.com is UP (Status: 200)
🔌 https://another-website.com is unreachable (Connection Error)
🕒 Waiting 5.0 minutes before next check...

